# Цены на продукты питания

In [62]:
import numpy as np
import pandas as pd
import scipy
import statsmodels
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
import plotly
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
import chart_studio.plotly as py
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import docx
import datetime as dt
import pickle
import random
import math
import time
import os
import re
import colorsys
import arrow
import locale
import requests
import collections
import numbers
import decimal

from warnings import simplefilter

from scipy import stats
from scipy.interpolate import make_interp_spline, BSpline
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from matplotlib.lines import Line2D
import matplotlib.dates as mdates

In [63]:
path_current = os.getcwd()
path_main = os.path.sep.join(path_current.split(os.path.sep)[:-2])

In [64]:
path_current

'/Users/achasovsky/Library/CloudStorage/GoogleDrive-alexanderchasovsky@gmail.com/Мой диск/Data Science Projects/# economy-rus-dash/notebooks/data-prepare'

In [65]:
path_main

'/Users/achasovsky/Library/CloudStorage/GoogleDrive-alexanderchasovsky@gmail.com/Мой диск/Data Science Projects/# economy-rus-dash'

### To main directory

In [66]:
# change directory to main path
os.chdir(path_main)

### Load libraries, functions, palette, theme

In [67]:
# load libraries and palette
%run __theme.ipynb

In [68]:
# load functions
# from __functions import *
%run __functions.ipynb

In [69]:
# start stopwatch
start = stopwatch_start()

### Variables

In [70]:
path_img = 'docs/img/'

In [71]:
path_files = path_main + '/files'

In [72]:
path_excel_csv = path_main + '/excel-csv'

### Load Files

In [73]:
economics_data = loadit('economics_data', path=path_files, create_empty_dict=True)

### Back to notebook directory

In [74]:
# back to current path
os.chdir(path_current)

# Цены на продукты питания

In [75]:
prices_producers_agriculture = economics_data['prices_producers_agriculture'].copy()

In [76]:
prices_consumers = economics_data['prices_consumers'].copy()

В 2022 году изменена номенклатура для хлеба.  

В 2021:
- хлеб и булочные изделия из пшеничной муки 1 и 2 сортов, кг
- хлеб и булочные изделия из пшеничной муки высшего сорта, кг


В 2022:
- хлеб и булочные изделия из пшеничной муки различных сортов, кг

В конечном итоге, цена для 2021 года считается как средняя от  'хлеб и булочные изделия из пшеничной муки 1 и 2 сортов, кг' 
и 'хлеб и булочные изделия из пшеничной муки высшего сорта, кг'. На графике цена за 2021 год обозначены светлым тоном.

In [77]:
prices_producers_columns = [
    'пшеница', 'крупный рогатый скот', 'свиньи', 'птица сельскохозяйственная живая',
    'молоко сырое крупного рогатого скота', 'овощи', 'яйца куриные в скорлупе свежие'
]

In [78]:
prices_consumers_columns = [
    'хлеб и булочные изделия из пшеничной муки 1 и 2 сортов, кг', 
    'хлеб и булочные изделия из пшеничной муки высшего сорта, кг',
    'хлеб и булочные изделия из пшеничной муки различных сортов, кг',
    'говядина (кроме бескостного мяса), кг', 'говядина бескостная, кг',
    'свинина (кроме бескостного мяса), кг', 'свинина бескостная, кг',
    'куры охлажденные и мороженые, кг', 
    'молоко питьевое цельное пастеризованное 2,5-3,2% жирности, л',
    'сыры сычужные твердые и мягкие, кг', 'сыры твердые, полутвердые и мягкие, кг',
    'сметана, кг', 'йогурт, 125 г', 'йогурт, кг', 'масло сливочное, кг', 
    'творог нежирный, кг', 'творог жирный, кг',
    'картофель, кг', 'огурцы свежие, кг', 'помидоры свежие, кг',
    'лук репчатый, кг', 'морковь, кг',
    'яйца куриные, 10 шт.'
]

In [79]:
prices_producers_food = prices_producers_agriculture[prices_producers_columns].copy()

In [80]:
prices_consumers_food = prices_consumers[prices_consumers_columns].copy()

## Цены - абсолютные

In [81]:
prices_food = pd.DataFrame()

### Хлеб

##### Производители

In [82]:
# 0.001 - tonns to kilos
prices_food = pd.concat(
    [prices_food, prices_producers_food['пшеница'] * 0.001], axis=1
)

##### Потребители

In [83]:
bread_2021 = pd.concat(
    [prices_consumers_food['хлеб и булочные изделия из пшеничной муки 1 и 2 сортов, кг'],
    prices_consumers_food['хлеб и булочные изделия из пшеничной муки высшего сорта, кг']],
    axis=1
)
bread_2021 = bread_2021.mean(axis=1)

In [84]:
bread = pd.concat([
    bread_2021.dropna(),
    prices_consumers_food['хлеб и булочные изделия из пшеничной муки различных сортов, кг'].dropna()
])

In [85]:
bread.name = 'хлеб и булочные изделия из пшеничной муки различных сортов, кг'

In [86]:
prices_food = pd.concat(
    [prices_food, bread], axis=1
)

### Говядина

##### Производители

In [87]:
# 0.001 - tonns to kilos
prices_food = pd.concat(
    [prices_food, prices_producers_food['крупный рогатый скот'] * 0.001], axis=1
)

##### Потребители

In [88]:
prices_food = pd.concat(
    [prices_food, prices_consumers['говядина (кроме бескостного мяса), кг']], axis=1
)

In [89]:
prices_food = pd.concat(
    [prices_food, prices_consumers['говядина бескостная, кг']], axis=1
)

### Свинина

##### Производители

In [90]:
# 0.001 - tonns to kilos
prices_food = pd.concat(
    [prices_food, prices_producers_food['свиньи'] * 0.001], axis=1
)

##### Потребители

In [91]:
prices_food = pd.concat(
    [prices_food, prices_consumers['свинина (кроме бескостного мяса), кг']], axis=1
)

In [92]:
prices_food = pd.concat(
    [prices_food, prices_consumers['свинина бескостная, кг']], axis=1
)

### Курица

##### Производители

In [93]:
# 0.001 - tonns to kilos
prices_food = pd.concat(
    [prices_food, prices_producers_food['птица сельскохозяйственная живая'] * 0.001], axis=1
)

##### Потребители

In [94]:
prices_food = pd.concat(
    [prices_food, prices_consumers['куры охлажденные и мороженые, кг']], axis=1
)

### Молоко

##### Производители

In [95]:
# 0.001 - tonns to kilos
prices_food = pd.concat(
    [prices_food, prices_producers_food['молоко сырое крупного рогатого скота'] * 0.001], axis=1
)

##### Потребители

In [96]:
prices_food = pd.concat(
    [prices_food, prices_consumers['молоко питьевое цельное пастеризованное 2,5-3,2% жирности, л']], axis=1
)

### Сыр

##### Потребители

In [97]:
cheese = pd.concat([
    prices_consumers['сыры сычужные твердые и мягкие, кг'].dropna(),
    prices_consumers['сыры твердые, полутвердые и мягкие, кг'].dropna()
])

In [98]:
cheese.name = 'сыры твердые, полутвердые и мягкие, кг'

In [99]:
prices_food = pd.concat(
    [prices_food, cheese], axis=1
)

### Сметана

##### Потребители

In [100]:
prices_food = pd.concat(
    [prices_food, prices_consumers['сметана, кг']], axis=1
)

### Йогурт

##### Потребители

In [101]:
jogurt = pd.concat([
    (prices_consumers['йогурт, 125 г'] / 0.125).dropna(),
    prices_consumers['йогурт, кг'].dropna()
])

In [102]:
jogurt.name = 'йогурт, кг'

In [103]:
prices_food = pd.concat(
    [prices_food, jogurt], axis=1
)

### Масло сливочное

##### Потребители

In [104]:
prices_food = pd.concat(
    [prices_food, prices_consumers['масло сливочное, кг']], axis=1
)

### Творог

В 2022 году изменилась номенклатура: 'творог жирный' и 'творог нежирный' заменем одним показателем - 'творог'.  
Данные для 2021 года считаются как среднее от 'творог жирный' и 'творог нежирный'.

##### Потребители

In [105]:
# mean of pd.Series using Numpy
cottage_cheese_2021 = pd.concat(
    [prices_consumers['творог нежирный, кг'],
     prices_consumers['творог жирный, кг']],
    axis=1).agg('mean', 1)

In [106]:
cottage_cheese = pd.concat([
    cottage_cheese_2021.dropna(),
    prices_consumers['творог, кг'].dropna()
])

In [107]:
cottage_cheese.name = 'творог, кг'

In [108]:
prices_food = pd.concat(
    [prices_food, cottage_cheese], axis=1
)

### Овощи

##### Производители

In [109]:
# 0.001 - tonns to kilos
prices_food = pd.concat(
    [prices_food, prices_producers_food['овощи'] * 0.001], axis=1
)

### Картофель

##### Потребители

In [110]:
prices_food = pd.concat(
    [prices_food, prices_consumers['картофель, кг']], axis=1
)

### Огурцы

##### Потребители

In [111]:
prices_food = pd.concat(
    [prices_food, prices_consumers['огурцы свежие, кг']], axis=1
)

### Помидоры

##### Потребители

In [112]:
prices_food = pd.concat(
    [prices_food, prices_consumers['помидоры свежие, кг']], axis=1
)

### Лук

##### Потребители

In [113]:
prices_food = pd.concat(
    [prices_food, prices_consumers['лук репчатый, кг']], axis=1
)

### Морковь

##### Потребители

In [114]:
prices_food = pd.concat(
    [prices_food, prices_consumers['морковь, кг']], axis=1
)

### Яйца куриные

##### Производители

In [115]:
# eggs price from 1000 eggs to 10 eggs
prices_food = pd.concat(
    [prices_food, prices_producers_food['яйца куриные в скорлупе свежие'] * 0.01], axis=1
)

##### Потребители

In [116]:
prices_food = pd.concat(
    [prices_food, prices_consumers['яйца куриные, 10 шт.']], axis=1
)

### Format data

In [117]:
prices_food.index = [to_date(i) for i in prices_food.index]

## Цены - динамика

In [118]:
prices_food_growth = pd.DataFrame()
for column in prices_food.columns:
    
    inter_data = normalized_by_first(prices_food[column])
    # inter_data = difference_by_first(inter_data.iloc[:, 0])
    # inter_data = (inter_data * 100)
    
    prices_food_growth = pd.concat([
        prices_food_growth, inter_data
    ], axis=1)

In [119]:
prices_food_growth = prices_food_growth.round(2)

### Save Dataset

In [120]:
economics_data['prices_food'] = prices_food

In [121]:
economics_data['prices_food_growth'] = prices_food_growth

In [122]:
saveit(
    file=economics_data,
    name='economics_data',
    path=path_files
)

File 'economics_data.pkl' saved


### Execution time

In [123]:
stopwatch_stop(start)

Execution time: 0:00:01
